In [1]:
import numpy as np
from qiskit.opflow import *
from qiskit.aqua.operators import SummedOp, PauliOp, MatrixOp
from qiskit.quantum_info import *

In [2]:
N = 5
num_turns = 2*(N - 1)
side_chain = [0]*N
lambda_back = 10

# GLOBAL
FULL_ID = I
for i in range(1, num_turns):
    FULL_ID = I^FULL_ID

In [3]:
def _create_pauli_for_conf(N):
    terms = []
    num_turns = 2*(N - 1)
    pauli_conf = np.zeros((num_turns, 2), dtype=object)

    for index in range(num_turns):
        if index != 0: 
            temp = I
        else: 
            temp = Z
        for i in range(1, num_turns):
            if i == index: 
                temp = Z^temp
            else:
                temp = I^temp
        terms.append(temp)
#     terms.reverse()
    
    for i in range(num_turns):
        pauli_conf[i][0] = terms[i]
        pauli_conf[i][1] = terms[i]   
    return pauli_conf

def _create_qubits_for_conf(pauli_conf):
    qubits = np.zeros(pauli_conf.shape, dtype=object)
    num_turns = qubits.shape[0]
    for i in range(num_turns):
        qubits[i][0] = (0.5*FULL_ID - 0.5*pauli_conf[i][0])
        qubits[i][1] = (0.5*FULL_ID - 0.5*pauli_conf[i][1])
    return qubits

# Create paulis for conformation

In [4]:
pauli_conf = _create_pauli_for_conf(N)

In [5]:
pauli_conf

array([[PauliOp(Pauli('IIIIIIIZ'), coeff=1.0),
        PauliOp(Pauli('IIIIIIIZ'), coeff=1.0)],
       [PauliOp(Pauli('IIIIIIZI'), coeff=1.0),
        PauliOp(Pauli('IIIIIIZI'), coeff=1.0)],
       [PauliOp(Pauli('IIIIIZII'), coeff=1.0),
        PauliOp(Pauli('IIIIIZII'), coeff=1.0)],
       [PauliOp(Pauli('IIIIZIII'), coeff=1.0),
        PauliOp(Pauli('IIIIZIII'), coeff=1.0)],
       [PauliOp(Pauli('IIIZIIII'), coeff=1.0),
        PauliOp(Pauli('IIIZIIII'), coeff=1.0)],
       [PauliOp(Pauli('IIZIIIII'), coeff=1.0),
        PauliOp(Pauli('IIZIIIII'), coeff=1.0)],
       [PauliOp(Pauli('IZIIIIII'), coeff=1.0),
        PauliOp(Pauli('IZIIIIII'), coeff=1.0)],
       [PauliOp(Pauli('ZIIIIIII'), coeff=1.0),
        PauliOp(Pauli('ZIIIIIII'), coeff=1.0)]], dtype=object)

# Create qubits for conformation

In [6]:
qubits = _create_qubits_for_conf(pauli_conf)

In [7]:
def _create_indic_turn(N, side_chain, qubits):
    if len(side_chain)!= N:
        raise Exception('size of side_chain list is not equal to N')
    num_turns = N - 1 
    indic_0 = np.zeros((num_turns, 2), dtype=object)
    indic_1 = np.zeros((num_turns, 2), dtype=object)
    indic_2 = np.zeros((num_turns, 2), dtype=object)
    indic_3 = np.zeros((num_turns, 2), dtype=object)
    r_conf = 0
    for i in range(num_turns):
        for m in range(2):
            if m == 1:
                if side_chain[i - 1] == 0:
                    continue
                else:
                    pass
            indic_0[i][m] = (FULL_ID - qubits[2*i][m])@(FULL_ID - qubits[2*i + 1][m])
            indic_1[i][m] = qubits[2*i + 1][m]@(qubits[2*i + 1][m] - 1*qubits[2*i][m])
            indic_2[i][m] = qubits[2*i][m]@(qubits[2*i][m] -1*qubits[2*i + 1][m])
            indic_3[i][m] = qubits[2*i][m]@(qubits[2*i + 1][m])
            r_conf += 1
    num_qubits = 2*r_conf - 5
    print('number of qubits required for conformation: ', num_qubits)
    return indic_0, indic_1, indic_2, indic_3, num_qubits


def _check_turns(i, p, j, s,
                 indic0, indic1, indic2,
                 indic3, pauli_conf):
#     t_ij = indic0[i][p]@indic0[j][s]
#     t_ij.add(indic1[i][p]@indic1[j][s])
#     t_ij.add(indic2[i][p]@indic2[j][s])
#     t_ij.add(indic3[i][p]@indic3[j][s]) 
    t_ij = indic0[i][p]@indic0[j][s] + indic1[i][p]@indic1[j][s] + \
           indic2[i][p]@indic2[j][s] + indic3[i][p]@indic3[j][s] 
#     t_ij = t_ij.reduce()
    return t_ij


def _create_H_back(N, lambda_back, indic_0,
                   indic_1, indic_2, indic_3,
                   pauli_conf):
    H_back = 0
    for i in range(N - 2):
        H_back += lambda_back*_check_turns(i, 0, i + 1, 0,
                                           indic_0, indic_1, indic_2, indic_3, pauli_conf)
    H_back = H_back.reduce()
    return H_back

In [8]:
indic_0, indic_1, indic_2, indic_3, num_qubits = _create_indic_turn(N, side_chain, qubits)
H_back = 0
H_back = _create_H_back(N, lambda_back, indic_0, indic_1, indic_2, indic_3, pauli_conf)

number of qubits required for conformation:  3


In [9]:
i = 2
j = 3
p = 0
s = 0
# t_ij = indic_0[i][p]@indic_0[j][s] + indic_1[i][p]@indic_1[j][s] + \
#            indic_2[i][p]@indic_2[j][s] + indic_3[i][p]@indic_3[j][s] 
t_ij = indic_3[i][p]@indic_3[j][s]
print(t_ij.reduce())

0.0625 * IIIIIIII
- 0.0625 * ZIIIIIII
- 0.0625 * IZIIIIII
+ 0.0625 * ZZIIIIII
- 0.0625 * IIZIIIII
+ 0.0625 * ZIZIIIII
+ 0.0625 * IZZIIIII
- 0.0625 * ZZZIIIII
- 0.0625 * IIIZIIII
+ 0.0625 * ZIIZIIII
+ 0.0625 * IZIZIIII
- 0.0625 * ZZIZIIII
+ 0.0625 * IIZZIIII
- 0.0625 * ZIZZIIII
- 0.0625 * IZZZIIII
+ 0.0625 * ZZZZIIII


In [10]:
for i in indic_3:
    print('Term is {}'.format(i[0]))

Term is 0.25 * IIIIIIII
- 0.25 * IIIIIIZI
- 0.25 * IIIIIIIZ
+ 0.25 * IIIIIIZZ
Term is 0.25 * IIIIIIII
- 0.25 * IIIIZIII
- 0.25 * IIIIIZII
+ 0.25 * IIIIZZII
Term is 0.25 * IIIIIIII
- 0.25 * IIZIIIII
- 0.25 * IIIZIIII
+ 0.25 * IIZZIIII
Term is 0.25 * IIIIIIII
- 0.25 * ZIIIIIII
- 0.25 * IZIIIIII
+ 0.25 * ZZIIIIII


In [11]:
# first_binaries = [1, -1, 1, 1, 0, -1]
# for k in [1, 2, 3, 4, 6]:
#     print('Pauli to replace at k = {} is {}'.format(k,first_binaries[k-1]))

In [12]:
# # need to update indic turns based on pre-set binaries
# indic = indic_3.copy()
# indic_updated = indic_3.copy()
# num_turns = N - 1
# new_tables = []
# new_coeffs = []

# for i in range(num_turns):
#         coeffs = np.copy(indic[i][0].primitive.coeffs[0])
#         if table_Z[1] == np.bool_(True):
#             coeffs = -1*coeffs
#         if table_Z[5] == np.bool_(True):
#             coeffs = -1*coeffs
# for i in range(num_turns):
#     for j in range(num_turns):
#         table_Z = np.copy(indic[i][0].primitive[j].table.Z[0])
#         table_X = np.copy(indic[i][0].primitive[j].table.X[0])
#         # get coeffs and update 
#         coeffs = np.copy(indic[i][0].primitive[j].coeffs[0])
#         if table_Z[1] == np.bool_(True):
#             coeffs = -1*coeffs
#         if table_Z[5] == np.bool_(True):
#             coeffs = -1*coeffs
#         # impose preset binary values
#         table_Z[0] = np.bool_(False)
#         table_Z[1] = np.bool_(False)
#         table_Z[2] = np.bool_(False)
#         table_Z[3] = np.bool_(False)
#         table_Z[5] = np.bool_(False)
#         print(table_Z)
#         new_table = np.concatenate((table_X, table_Z), axis=0)
#         new_tables.append(new_table)
#         new_coeffs.append(coeffs)
#     new_pauli_table = PauliTable(data=new_tables)
#     indic_updated[i][0] = PauliSumOp(SparsePauliOp(data=new_pauli_table))

        
            
        

In [13]:
indic_3[0][0].primitive.coeffs

array([ 0.25+0.j, -0.25+0.j, -0.25+0.j,  0.25+0.j])

In [14]:
for H in H_back:
    print(H)

7.5 * IIIIIIII
2.5 * ZIZIIIII
2.5 * IZIZIIII
2.5 * ZZZZIIII
2.5 * IIZIZIII
2.5 * IIIZIZII
2.5 * IIZZZZII
2.5 * IIIIZIZI
2.5 * IIIIIZIZ
2.5 * IIIIZZZZ
